In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Portfolio Analysis").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
from pyspark.sql.functions import *

stock_lst = ["AAPL","ABBV","ABT","ADBE","ADP","AMGN","AMT","AMZN","AROW","ARTNA","AXP","AZN","BA", \
             "BABA","BAC","BCAL","BHP","BKNG","BTI","CAT","CECO","CHEF","CL","CMRX","COCO","CRM", \
             "CRML","CSCO","CTOS", "CVX","DEO","DIS","ELLO","EQIX","ETN","FAT","GCMG","GE","GOOGL", \
             "GRC","GS","GWRS","HIMX","HIPO","HON","IBM","ICHR","ISRG","JNJ","JPM","KLG","KMB","KO", \
             "LIN","LLY","LMT","MA","MCD","MDLZ","META","MS","MSFT","MXL","NFLX","NKE","NSPR","NTES", \
             "NVDA","ORCL","PCYO","PDD","PFE","PG","PLCE","PM","PNNT","PNST","RIO","SAP","SIRI","SNY", \
             "SPOT","SRG","SUP","TCI","TH","TJX","TM","TSLA", "TSM","UAN","UL","UNH","UNP","V","VZ","WFC", \
             "WMG","WMT","YTRA"]

stockdata =spark.sql("select * from stocks")

stockdata = stockdata.withColumn("date",to_date(stockdata.date,"yyyy-MM-dd"))



In [4]:
# stocks with closing price datewise
stockscp_df=stockdata.groupBy("date").pivot("symbol", stock_lst).sum("adjClose")
#stockscp_df=stockscp_df.dropna()
stockscp_df.show()

+----------+------+------+------+----------+------+------+------+------+-----+-----+------+-----+----------+------+-----+---------+-----+-------+-----+------+-----+-----+-----+------+---------+------+-----+-----+----+------+------+------+---------+------+------+-----+----+------+------+-----+------+-----+-----+---------+------+------+---------+------+------+------+-----+------+-----+------+------+------+------+------+-----+------+-----+------+-----+----------+------+-----+------+------+------+---------+----------+-----+------+---------+------+----+-----+-----+------+----+-----+----------+---------+----+---------+------+------+------+------+------+-----+-----+------+------+------+-----+-----+-----+-----+-----+
|      date|  AAPL|  ABBV|   ABT|      ADBE|   ADP|  AMGN|   AMT|  AMZN| AROW|ARTNA|   AXP|  AZN|        BA|  BABA|  BAC|     BCAL|  BHP|   BKNG|  BTI|   CAT| CECO| CHEF|   CL|  CMRX|     COCO|   CRM| CRML| CSCO|CTOS|   CVX|   DEO|   DIS|     ELLO|  EQIX|   ETN|  FAT|GCMG|    GE| 

In [5]:
filepath='file:////home/talentum/shared/Project/Project_codes/source/SP500.csv'

stockdata1=spark.read.option("header",True).csv(filepath)
stockdata1 = stockdata1.withColumn("date",to_date(stockdata1.Date,"MM/dd/yyyy"))
stockdata1=stockdata1.dropna()

In [6]:
#SP500 with closing price datewise
sp500_df=stockdata1.select("date","Close")
sp500_df=sp500_df.withColumnRenamed("Close","SP500")
sp500_df=sp500_df.dropna()
sp500_df.show()

+----------+-------+
|      date|  SP500|
+----------+-------+
|2024-08-09|5344.16|
|2024-08-08|5319.31|
|2024-08-07|5199.50|
|2024-08-06|5240.03|
|2024-08-05|5186.33|
|2024-08-02|5346.56|
|2024-08-01|5446.68|
|2024-07-31|5522.30|
|2024-07-30|5436.44|
|2024-07-29|5463.54|
|2024-07-26|5459.10|
|2024-07-25|5399.22|
|2024-07-24|5427.13|
|2024-07-23|5555.74|
|2024-07-22|5564.41|
|2024-07-19|5505.00|
|2024-07-18|5544.59|
|2024-07-17|5588.27|
|2024-07-16|5667.20|
|2024-07-15|5631.22|
+----------+-------+
only showing top 20 rows



In [8]:
#Creating returns Dataframe

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

returns_df_sp500=stockscp_df.join(sp500_df,"date")
returns_df_sp500.show()
my_window = Window.partitionBy().orderBy("date")
for col_name in returns_df_sp500.columns:
    if col_name!='date':
        returns_df_sp500 = returns_df_sp500.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0) \
                                                       .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in returns_df_sp500.columns:
    if c!='date':
        returns_df_sp500 = returns_df_sp500.withColumn(c, F.round(c, 6))

returns_df_sp500=returns_df_sp500.dropna()
returns_df_sp500=returns_df_sp500.drop('date')
returns_df_sp500.show()

returns_df=returns_df_sp500.drop("SP500")
returns_df.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------

In [9]:
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [10]:
def stocks_correlation(stocks_df,cols):
    # convert to vector column first
    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=cols, outputCol=vector_col)
    stocks_df_vector = assembler.transform(stocks_df).select(vector_col)

    # get correlation matrix
    matrix = Correlation.corr(stocks_df_vector, vector_col).collect()[0][0]
    corrmatrix = matrix.toArray().tolist()

    corrmatrix_df = spark.createDataFrame(corrmatrix,cols)

    for c in corrmatrix_df.columns:
        corrmatrix_df = corrmatrix_df.withColumn(c,F.round(c,2))

    stock_list = corrmatrix_df.columns #["AAPL","KO","NFLX","DIS","IBM","VZ","WMT","GE","TSLA","MA","AMZN","MSFT","UN","V"]

    # Create a DataFrame from the list of values
    values_df = spark.createDataFrame([Row(new_column=v) for v in stock_list])

    corrmatrix_df = corrmatrix_df.withColumn("row_index", monotonically_increasing_id())
    values_df = values_df.withColumn("row_index", monotonically_increasing_id())

    # Join the DataFrames on the row index
    corr_df = corrmatrix_df.join(values_df, on="row_index").drop("row_index")

    # Show the resulting DataFrame
    corr_df=corr_df.withColumnRenamed("new_column","stockName")
    return corr_df

In [11]:
corr_final_df = stocks_correlation(returns_df,returns_df.columns)
corr_final_df.show()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------+
| AAPL| ABBV|  ABT| ADBE|  ADP| AMGN|  AMT| AMZN| AROW|ARTNA|  AXP|  AZN|   BA|BABA|  BAC| BCAL|  BHP| BKNG|  BTI|  CAT| CECO| CHEF|   CL| CMRX| COCO|  CRM| CRML| CSCO| CTOS|  CVX|  DEO|  DIS| ELLO| EQIX|  ETN|  FAT| GCMG|   GE|GOOGL|  GRC|   GS| GWRS| HIMX| HIPO|  HON|  IBM| ICHR| ISRG|  JNJ|  JPM|  KLG|  KMB|   KO|  LIN|  LLY|  LMT|   MA|  MCD| MDLZ| META|   MS| MSFT|  MXL| NFLX|  NKE|

In [12]:
def highmoderateCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.5 and 0.99".format(st) # moderate -0.5 to 0.69 & high 0.7 to 0.99
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [13]:
correlated_portfolios = highmoderateCorrelation(corr_final_df,stock_lst)

In [14]:
correlated_portfolios

In [14]:
portfolios = []
for port in correlated_portfolios:
    if 5 < len(port) <= 10:
        portfolios.append(port)

In [15]:
portfolios

[['SIRI', 'TSLA', 'AMT', 'JNJ', 'SAP', 'ABBV', 'IBM', 'EQIX', 'ADBE'],
 ['UNH', 'AMT', 'KO', 'JNJ', 'NSPR', 'ABBV', 'BTI', 'LMT', 'ADP'],
 ['SIRI', 'ICHR', 'CTOS', 'DEO', 'TCI', 'CMRX', 'EQIX', 'PLCE', 'NTES', 'BA'],
 ['CRM', 'DIS', 'TH', 'HIPO', 'TM', 'CSCO', 'YTRA', 'CAT'],
 ['TH', 'UAN', 'ETN', 'SUP', 'WFC', 'KLG', 'BHP', 'RIO', 'CVX'],
 ['CVX', 'TH', 'UAN', 'HIPO', 'ETN', 'SUP', 'RIO', 'KLG'],
 ['AMZN', 'ICHR', 'TM', 'WMG', 'BHP', 'MXL'],
 ['UNH', 'UNP', 'JNJ', 'AROW', 'GRC', 'BCAL', 'GWRS', 'PCYO'],
 ['CAT', 'CVX', 'UAN', 'HIPO', 'SUP', 'YTRA', 'KLG', 'RIO', 'TH'],
 ['TSM', 'TJX', 'JPM', 'ORCL', 'ISRG', 'CECO', 'LLY', 'VZ']]

In [15]:
#portfolios=[ ['AAPL', 'DIS', 'TSLA', 'VZ'],['MSFT', 'ISRG', 'META']]

In [16]:
#portfolios

[['AAPL', 'DIS', 'TSLA', 'VZ'], ['MSFT', 'ISRG', 'META']]

In [17]:
port_df = spark.createDataFrame([(row,) for row in portfolios], ["values"])
port_df_exploded = port_df.select(explode(col("values")).alias("value"))
unique_port_df = port_df_exploded.distinct()
unique_port =unique_port_df.rdd.flatMap(lambda x :x).collect()

In [18]:
unique_port

['AAPL', 'META', 'DIS', 'TSLA', 'VZ', 'ISRG', 'MSFT']

In [19]:
# Names to Portfolios
names = []

for i in range(1, len(portfolios)+1):
    names.append('Portfolio'+str(i))


In [20]:
names

['Portfolio1', 'Portfolio2']

In [21]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
from pyspark.mllib.linalg import DenseMatrix

"""
Returns the Annualised Expected Return of a portfolio.
Annualises the return using the 'crude' method.
"""
def getPortReturns():
    portfolio_returns = []
    for port,weight in zip(portfolios,portfolio_weightage):
           
        num_stocks=len(port)
        # creating weight matrix  
        weight_matrix= DenseMatrix(1,num_stocks, weight)
        # Creating Row matrix for the Dense matrix to do dot_product using multiply function
        weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
        # Extracting mean return of the portfolio
        returns_df_mean_list1=[]
        returns_df_mean_list=[]
        for p in port:
            returns_df_mean_list = returns_df_mean.filter(returns_df_mean.Stocks == p).select("Mean").rdd.flatMap(lambda x: x).collect()
            returns_df_mean_list1.append(returns_df_mean_list[0])
                
        returns_df_matrix=DenseMatrix(len(returns_df_mean_list1),1,returns_df_mean_list1)
        # calculating dot_product of weight and mean return to get the expected return
        exp_ret_portfolio=weight_row_mat.multiply(returns_df_matrix).rows.collect()
        # annual expected return through crude method
        exp_ret_portfolio_annual = exp_ret_portfolio[0][0] * 250
        # Returning the lsit of expected return of portfolios
        portfolio_returns.append(exp_ret_portfolio_annual)
    
    return portfolio_returns
    
    
    
    

In [22]:
# Calculate the risk of each portfolio
from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
import numpy as np
# arguments are weight and return_df of portfolios #df_temp = return_df[port]
def getPortRisk(weights,df_temp):
    portfolio_risks = []
    
    # calculating the variance covariance matrix of return_df of portfolio
    
    returns_rdd = df_temp.rdd.map(list)
    returns_row_mat = RowMatrix(returns_rdd)
    vcv_matrix=returns_row_mat.computeCovariance()
   
    # Variance of the portfolio = weight matrix dot_product variance-covariance matricx dot_product weight transpose matrix
    weight_matrix= DenseMatrix(1, num_stocks, weights) # weight matrix
    weight_matrix_t= DenseMatrix(num_stocks, 1, weights) # weight matrix Transpose
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    vcv_w=weight_row_mat.multiply(vcv_matrix) # dot_product of weight matrix and vcv matrix
    var_portfolio = vcv_w.multiply(weight_matrix_t).rows.collect() # dot_product of (weight matrix and vcv matrix in prev line) with weight transpose
    
    # calculating standard deviation from variance and compute annual sd of the portfolio
    var_portfolio=var_portfolio[0][0]
    sd_portfolio=np.sqrt(var_portfolio)
    sd_portfolio_annual = sd_portfolio * np.sqrt(250)
    
    # returning the portfolio risk i.e. standard deviation computed annually
    portfolio_risks.append(sd_portfolio_annual)
    
    return portfolio_risks

In [23]:
returns_df_unique=returns_df[unique_port]

In [24]:
returns_df_unique.show()

+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|     META|      DIS|     TSLA|       VZ|     ISRG|     MSFT|
+---------+---------+---------+---------+---------+---------+---------+
|-0.010048|-0.004324|  9.33E-4|-0.027557|-0.001775| 0.001773|-0.003188|
|-0.007461|-0.004736| -5.59E-4| 0.038704|-0.023876|-0.003848|-0.006812|
| 0.008126| 0.011034|  0.01614| 0.001655| 0.005985|-0.004662| -1.47E-4|
|-0.006634|-0.006227| 0.012578| 0.011566| 0.004397|  3.36E-4|   5.9E-4|
|-0.003643| 0.012615| 0.007072| -7.92E-4| -0.00206|-0.002561| 0.014521|
|-0.006038| 0.024835| 0.003331| 0.003765| 0.004387| 0.031898| 0.004481|
|-0.025415|-0.008171| 0.015524|-0.071555| 0.002826|  0.00676|-0.001398|
|-0.028423|-0.016014|-0.007246|-0.039332| 0.001025|-0.033126|-0.029553|
| 0.010251|-0.012176| 0.001277|-0.018528| -5.23E-4|-0.002114|-0.007125|
| 0.011848|-0.044198| 0.018031| 0.013859| 0.015686|-0.033048|-0.004192|
| 0.002782| 0.033409| 0.001342|-0.001294|      0.0| 0.027712| 0.

In [25]:
# Calculating the weightage of the stocks for the portfolio by minimising risks
from scipy.optimize import minimize
import numpy as np

portfolio_weightage=[]
for port in portfolios:

    num_stocks = len(returns_df_unique[port].columns)  # being the number of stocks (this is a 'global' variable)
    init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0) [Intially assume equal weightge]
    args=returns_df_unique[port] # second argument of getPortRisk() function
    bounds = tuple((0, 1) for i in range(num_stocks)) # weightage will be between 0 and 1
    cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1}) # sum of weightage will be 1
    # getting the optimised weightage through minimise() function with getPortRisk() as objective function and applying bounadries and constraints as adefined above
    results = minimize(fun=getPortRisk, x0=init_weights,args=args, bounds=bounds, constraints=cons)
    # converting the values to float and rounding off to 2 decimal places
    rounded_result = np.round(results.x, decimals=2)
    print(rounded_result)
    temp_rdd=sc.parallelize(rounded_result)
    temp_rdd=temp_rdd.map(lambda x:float(x))
    
    portfolio_weightage.append(temp_rdd.collect())


[0.24 0.16 0.   0.6 ]
[0.64 0.36 0.  ]


In [26]:
portfolio_weightage

[[0.24, 0.16, 0.0, 0.6], [0.64, 0.36, 0.0]]

In [27]:
# Calculating the investment amount of the portfolio i.e. latest closing price

latest_date = stockscp_df.select(max("date").alias("latest_date")).collect()[0]["latest_date"]
portfolio_amount = []

for port in portfolios:
    amount_rdd=stockscp_df.filter(stockscp_df.date == latest_date).select(port).rdd.flatMap(lambda x: x).map(lambda x: float(x))
    total_amount= amount_rdd.collect()
    portfolio_amount.append(total_amount)
    
    

In [28]:
print(latest_date)

2024-08-09


In [29]:
portfolio_amount

[[216.24, 86.21, 200.0, 40.75], [406.02, 463.425, 517.77]]

In [30]:
# calculating mean return for each stocks and creating a dtaframe
data1=unique_port
data2=[]
for c in unique_port:
    data2.append(returns_df.select(mean(col(c))).collect()[0][0]) 
data = list(zip(data1, data2))
returns_df_mean=spark.createDataFrame(data,["Stocks","Mean"])
returns_df_mean = returns_df_mean.withColumn("Mean", round(returns_df_mean["Mean"], 6))
returns_df_mean.show()

+------+---------+
|Stocks|     Mean|
+------+---------+
|  AAPL| 0.002978|
|  META| 0.001559|
|   DIS|-0.001526|
|  TSLA|   0.0019|
|    VZ| 0.001893|
|  ISRG| 0.001286|
|  MSFT|  9.33E-4|
+------+---------+



In [31]:
# function call for returns calulation as per thw portflio weightage
portfolio_return_wt=getPortReturns()


In [32]:
portfolio_return_wt

[0.40159, 0.26502000000000003]

In [33]:
# Calculating beta_value with Linear Regression 

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=["SP500"], outputCol="features")
data_with_features = assembler.transform(returns_df_sp500)
betaValues=[]
for col in returns_df_unique.columns:
# Create a LinearRegression model and fit it to the data
    lr = LinearRegression(featuresCol="features", labelCol=col) # passing each stock to LR as labelCol
    model = lr.fit(data_with_features)
    betaValues.append(model.coefficients[0]) # model.coefficients[0] gives beta value of specific stock 

print(betaValues)


[1.0218643420684215, 1.6184747438464004, 0.7483664400790094, 2.3745145794419997, 0.1171975674678391, 1.1551767003758897, 1.1826763759398278]


In [34]:
#Creating dataframe for Stocks with beta value
d1=unique_port
d2=[float(x) for x in betaValues]
d=zip(d1,d2)
beta_df=spark.createDataFrame(d,["Stocks","Beta"])
beta_df=beta_df.withColumn("Beta",round("Beta",2))
beta_df.show()

+------+----+
|Stocks|Beta|
+------+----+
|  AAPL|1.02|
|  META|1.62|
|   DIS|0.75|
|  TSLA|2.37|
|    VZ|0.12|
|  ISRG|1.16|
|  MSFT|1.18|
+------+----+



In [35]:
# Grouping the beta values for the portfolio

beta_port=[]
for port in portfolios:
    beta_list=[]
    for p in port:
        beta_rdd=beta_df.filter(beta_df.Stocks == p).select("Beta").rdd.flatMap(lambda x: x).map(lambda x: float(x))
        beta_list.append(beta_rdd.collect())
        beta_list_rdd=sc.parallelize(beta_list)
        beta_list_f = beta_list_rdd.flatMap(lambda x: x).collect()
    beta_port.append(beta_list_f)
        
print(beta_port)

[[1.02, 0.75, 2.37, 0.12], [1.18, 1.16, 1.62]]


In [36]:
# Calculating the final Portfolio Dataframe incorporating Names, Portfolios, Weightage,return,Amount,Beta values

portfolio_return_wt=[float(x) for x in portfolio_return_wt]

data=list(zip(names,portfolios,portfolio_weightage,portfolio_return_wt,portfolio_amount,beta_port))
schema = StructType([
    StructField("Names", StringType(), True),
    StructField("Portfolios", ArrayType(StringType()), True),
    StructField("Weightage", ArrayType(FloatType()), True),
    StructField("TotalReturns", FloatType(), True),
    StructField("Amount", ArrayType(FloatType()), True),
    StructField("BetaPort", ArrayType(FloatType()), True)])
                
final_portfolio_df = spark.createDataFrame(data, schema)

final_portfolio_df.show()

+----------+--------------------+--------------------+------------+--------------------+--------------------+
|     Names|          Portfolios|           Weightage|TotalReturns|              Amount|            BetaPort|
+----------+--------------------+--------------------+------------+--------------------+--------------------+
|Portfolio1|[AAPL, DIS, TSLA,...|[0.24, 0.16, 0.0,...|     0.40159|[216.24, 86.21, 2...|[1.02, 0.75, 2.37...|
|Portfolio2|  [MSFT, ISRG, META]|   [0.64, 0.36, 0.0]|     0.26502|[406.02, 463.425,...|  [1.18, 1.16, 1.62]|
+----------+--------------------+--------------------+------------+--------------------+--------------------+



In [37]:
# Calculating the Investment amount and beta value as per the  weightage

final_portfolio_df = final_portfolio_df.withColumn("InvestmentAmount", expr("aggregate(zip_with(Weightage, Amount, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("Amount")

final_portfolio_df = final_portfolio_df.withColumn("PortfolioBeta", expr("aggregate(zip_with(Weightage, BetaPort, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("BetaPort")

final_portfolio_df = final_portfolio_df.withColumn("TotalReturns", round(final_portfolio_df["TotalReturns"], 2)) \
.withColumn("InvestmentAmount", round(final_portfolio_df["InvestmentAmount"], 2)) \
.withColumn("PortfolioBeta", round(final_portfolio_df["PortfolioBeta"], 2))

final_portfolio_df.show(truncate=False)

+----------+---------------------+----------------------+------------+----------------+-------------+
|Names     |Portfolios           |Weightage             |TotalReturns|InvestmentAmount|PortfolioBeta|
+----------+---------------------+----------------------+------------+----------------+-------------+
|Portfolio1|[AAPL, DIS, TSLA, VZ]|[0.24, 0.16, 0.0, 0.6]|0.4         |90.14           |0.44         |
|Portfolio2|[MSFT, ISRG, META]   |[0.64, 0.36, 0.0]     |0.27        |426.69          |1.17         |
+----------+---------------------+----------------------+------------+----------------+-------------+



In [38]:
# Incorporate Volatility based on beta value , Beta VAlue > 1 High Volatile else Less Volatile

from pyspark.sql import functions as F

final_portfolio_df = final_portfolio_df.withColumn("Volatility", F.when(final_portfolio_df["PortfolioBeta"] > 1, "High") \
                                                   .otherwise("Low"))
final_portfolio_df=final_portfolio_df.drop("PortfolioBeta")
final_portfolio_df.show()




+----------+--------------------+--------------------+------------+----------------+----------+
|     Names|          Portfolios|           Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+--------------------+--------------------+------------+----------------+----------+
|Portfolio1|[AAPL, DIS, TSLA,...|[0.24, 0.16, 0.0,...|         0.4|           90.14|       Low|
|Portfolio2|  [MSFT, ISRG, META]|   [0.64, 0.36, 0.0]|        0.27|          426.69|      High|
+----------+--------------------+--------------------+------------+----------------+----------+



In [39]:
df_exploded1 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility",posexplode("Portfolios").alias("pos", "Portfolios"))
df_exploded2 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility", posexplode("Weightage").alias("pos", "Weightage"))

df_final = df_exploded1.join(df_exploded2, on=["Names","TotalReturns","InvestmentAmount","Volatility", "pos"]) \
.select("Names", "Portfolios", "Weightage","TotalReturns","InvestmentAmount","Volatility")

df_sorted = df_final.sort("Names")
df_sorted.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|      AAPL|     0.24|         0.4|           90.14|       Low|
|Portfolio1|        VZ|      0.6|         0.4|           90.14|       Low|
|Portfolio1|       DIS|     0.16|         0.4|           90.14|       Low|
|Portfolio1|      TSLA|      0.0|         0.4|           90.14|       Low|
|Portfolio2|      ISRG|     0.36|        0.27|          426.69|      High|
|Portfolio2|      META|      0.0|        0.27|          426.69|      High|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|
+----------+----------+---------+------------+----------------+----------+



In [40]:
df_sorted_fin=df_sorted.filter(df_sorted.Weightage != 0)

In [41]:
df_sorted_fin.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|       DIS|     0.16|         0.4|           90.14|       Low|
|Portfolio1|        VZ|      0.6|         0.4|           90.14|       Low|
|Portfolio1|      AAPL|     0.24|         0.4|           90.14|       Low|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|
|Portfolio2|      ISRG|     0.36|        0.27|          426.69|      High|
+----------+----------+---------+------------+----------------+----------+



In [42]:
filepath='file:////home/talentum/shared/Project/Project_codes/source/company_names.csv'

company_df=spark.read.option("header",True).csv(filepath)

In [43]:
company_df.show()

+------+--------------------+
|symbol|             company|
+------+--------------------+
|  AAPL|          Apple Inc.|
|  ABBV|         AbbVie Inc.|
|   ABT| Abbott Laboratories|
|  ADBE|          Adobe Inc.|
|   ADP|Automatic Data Pr...|
|  AMGN|          Amgen Inc.|
|   AMT|American Tower Co...|
|  AMZN|    Amazon.com, Inc.|
|  AROW|Arrow Financial C...|
| ARTNA|Artesian Resource...|
|   AXP|American Express ...|
|   AZN|     AstraZeneca PLC|
|    BA|  The Boeing Company|
|  BABA|Alibaba Group Hol...|
|   BAC|Bank of America C...|
|  BCAL|Southern Californ...|
|   BHP|   BHP Group Limited|
|  BKNG|Booking Holdings ...|
|   BTI|British American ...|
|   CAT|    Caterpillar Inc.|
+------+--------------------+
only showing top 20 rows



In [44]:
df_fin=df_sorted_fin.join(company_df,df_sorted_fin.Portfolios==company_df.symbol,"inner")
df_fin=df_fin.drop("symbol")
df_fin.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|      AAPL|     0.24|         0.4|           90.14|       Low|          Apple Inc.|
|Portfolio1|        VZ|      0.6|         0.4|           90.14|       Low|Verizon Communica...|
|Portfolio1|       DIS|     0.16|         0.4|           90.14|       Low|The Walt Disney C...|
|Portfolio2|      ISRG|     0.36|        0.27|          426.69|      High|Intuitive Surgica...|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|
+----------+----------+---------+------------+----------------+----------+--------------------+



In [45]:
df_fin.write.mode("overwrite").saveAsTable("Portfolios")

In [46]:
portfoliodata=spark.sql("select * from Portfolios")

In [47]:
portfoliodata.show()
stockdata.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|      AAPL|     0.24|         0.4|           90.14|       Low|          Apple Inc.|
|Portfolio1|        VZ|      0.6|         0.4|           90.14|       Low|Verizon Communica...|
|Portfolio1|       DIS|     0.16|         0.4|           90.14|       Low|The Walt Disney C...|
|Portfolio2|      ISRG|     0.36|        0.27|          426.69|      High|Intuitive Surgica...|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|
+----------+----------+---------+------------+----------------+----------+--------------------+

+------+--------+-------+--------------+-------------+-------+----------+-------+-------------+-------+-------+----------------+------+

In [48]:
joineddf = portfoliodata.join(stockdata, portfoliodata["Portfolios"] == stockdata["symbol"], "inner")
joineddf=joineddf.drop("label")
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+------+----------+------+------+------+----------------+--------+--------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose|change|changeOverTime|changePercent| close|      date|  high|   low|  open|unadjustedVolume|  volume|    vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+------+----------+------+------+------+----------------+--------+--------+
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|  MSFT|  406.02|  1.99|     0.0049254|      0.49254|406.02|2024-08-09|408.05|402.26|404.03|        19018056|19018056|  405.09|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|  MSFT|  402.69| 

In [49]:
joineddf.write.mode("overwrite").saveAsTable("PortfoliosDetail")

In [50]:
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+------+----------+------+------+------+----------------+--------+--------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose|change|changeOverTime|changePercent| close|      date|  high|   low|  open|unadjustedVolume|  volume|    vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+------+----------+------+------+------+----------------+--------+--------+
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|  MSFT|  406.02|  1.99|     0.0049254|      0.49254|406.02|2024-08-09|408.05|402.26|404.03|        19018056|19018056|  405.09|
|Portfolio2|      MSFT|     0.64|        0.27|          426.69|      High|Microsoft Corpora...|  MSFT|  402.69| 

In [51]:
filepath='file:////home/talentum/csvfile_final'

#dffile=spark.sql("select * from PortfoliosDetail")
# Write DataFrame to CSV file with header
#sd.write.mode("overwrite").format("csv").save(filepath)

joineddf.coalesce(1).write.mode("overwrite").option("header","true").format("csv").save(filepath)